### 加载pair-wise数据集，返回一个list-wise的数据集，每个sample内包含5个文档，context全相同，label为是否点击

In [1]:
pairwise_path = "/home/zhaoheng_huang/CAGS_data/Rank/data/aol/train/train_line_pairwise.txt"
listwise_path = "/home/zhaoheng_huang/CAGS_data/Rank/data/aol/train/train_list.txt"

In [2]:
with open(pairwise_path, "r") as fp:
    pair_data = fp.readlines()
pair_data = list(map(lambda x: [y.split("\t") for y in x.strip().split("[####]")], pair_data))

In [3]:
def getNextIndex(List, curID, length):
    """
    获得下一组ongoing session的初始下标
    """
    session = List[curID][0]
    resID = curID + 1
    while resID < length and List[resID][0] == session:
        resID += 1
    return resID

In [4]:
def genListwise_aol(sessionPair: list) -> list:
    length = len(sessionPair)
    curID = 0
    res = []   # sample: {"clicksession": 1, "unclicksession": 0, ...}
    while curID < length:
        nextID = getNextIndex(sessionPair, curID, length)
        sample = {}
        for i in range(curID, nextID):
            sample["\t".join(sessionPair[i][1])] = 0
        sample["\t".join(sessionPair[curID][0])] = 1 # 已点击，覆盖掉未点击的结果
        sample = list(sorted(sample.items(), key=lambda x: -x[1]))
        if len(sample) == 1: pass   # navigation query
        else:
            appendSize = 5 - len(sample)
            for i in range(appendSize):
                k, v = sample[-1]
                sample.append((k, v))
            res.append(sample)
        curID = nextID
    return res

In [5]:
res = genListwise_aol(sessionPair = pair_data)

In [8]:
res[0]

[('mapquest com\tofficial mapquest maps driving directions live traffic', 1),
 ('mapquest com\tmapquest driving directions', 0),
 ('mapquest com\tmapquest developer network mapping encoding routes traffic mapquest developer network',
  0),
 ('mapquest com\tmapquest developer network mapping encoding routes traffic mapquest developer network',
  0),
 ('mapquest com\tmapquest developer network mapping encoding routes traffic mapquest developer network',
  0)]

### 数据集每一行被[####]分开成五个部分，其中第一个是点击的序列，剩下的是未点击

In [9]:
with open(listwise_path, "w") as fw:
    for line in res:
        # 按click和unclick分类存储
        writeline = []
        for seq, label in line:
            writeline.append(seq)
        writeline = "[####]".join(writeline)
        fw.write(writeline+"\n")

In [10]:
!head $listwise_path

mapquest com	official mapquest maps driving directions live traffic[####]mapquest com	mapquest driving directions[####]mapquest com	mapquest developer network mapping encoding routes traffic mapquest developer network[####]mapquest com	mapquest developer network mapping encoding routes traffic mapquest developer network[####]mapquest com	mapquest developer network mapping encoding routes traffic mapquest developer network
mapquest com	official mapquest maps driving directions live traffic	mapquest com	official mapquest maps driving directions live traffic[####]mapquest com	official mapquest maps driving directions live traffic	mapquest com	mapquest driving directions[####]mapquest com	official mapquest maps driving directions live traffic	mapquest com	how can we help mapquest help[####]mapquest com	official mapquest maps driving directions live traffic	mapquest com	discover the new mapquest[####]mapquest com	official mapquest maps driving directions live traffic	mapquest com	discover t

In [ ]:
!head 

In [51]:
res[:5]

[{'mapquest com\tofficial mapquest maps driving directions live traffic': 1,
  'mapquest com\tmapquest driving directions': 0,
  'mapquest com\tmapquest developer network mapping encoding routes traffic mapquest developer network': 0},
 {'mapquest com\tofficial mapquest maps driving directions live traffic\tmapquest com\tofficial mapquest maps driving directions live traffic': 1,
  'mapquest com\tofficial mapquest maps driving directions live traffic\tmapquest com\tmapquest driving directions': 0,
  'mapquest com\tofficial mapquest maps driving directions live traffic\tmapquest com\thow can we help mapquest help': 0,
  'mapquest com\tofficial mapquest maps driving directions live traffic\tmapquest com\tdiscover the new mapquest': 0},
 {'pentail pools com\tpentail high performance pool and spa equipment for pool perfection': 1,
  'pentail pools com\tpentail equipment training series': 0,
  'pentail pools com\tpentail aurora pump': 0,
  'pentail pools com\tpentail pentail fluid motion': 

In [43]:
pair_data[4]

[['mapquest com',
  'official mapquest maps driving directions live traffic',
  'mapquest com',
  'official mapquest maps driving directions live traffic'],
 ['mapquest com',
  'official mapquest maps driving directions live traffic',
  'mapquest com',
  'official mapquest maps driving directions live traffic']]